# 2021-1117 Zoho Email ZZbkp parsing 

    - backedup emails as .eml from zoho through 4 folders [ inbox, sent, business, operations ] 
    - rename these files by reading and parsing the files into a list and using the list features in rename 

## Import and Ufunc

In [358]:
import os
import pandas as pd 
import numpy as np 

## email pkg provides the core to parsing a message 
import email

## progress tracing pkg 
from tqdm import tqdm

## key library used for parsing a date like string into a python datetime value 
from dateutil import parser

In [421]:
def convert_date_2_string(dstr):
    dt_str = ""
    try:
        dt_str = parser.parse(dstr)

    except:
        for sx in [0,4]:
            for ex in range(12,18):
                try:
                    dt_str = parser.parse(dstr[sx,ex])
                except:
                    dt_str = dt_str
    return dt_str
    
## EOF :: ufunc to generate date from string, badly named !! 
## Ref :: https://stackoverflow.com/questions/466345/converting-string-into-datetime 


## Process Files 
### Key Variables 

In [422]:
root_path = "/home/gaya3/Downloads/Ubuntu WIN10VM/2021-1116 zzbkp zohomail sdsl vra"
dir_sent = "Sent/"
list_path = []
list_ff = []
# ff_list_sent = os.listdir(f"{root_path}{dir_sent}")


"""used os.walk to parse through sub folders, the outputs, 
comes as files list per subfolder therefore second nested tor loop needed """
" https://stackoverflow.com/questions/2909975/python-list-directory-subdirectory-and-files "
for path, subdirs, ffs in os.walk(root_path):
    for name in ffs:
        if ".eml" in name: 
            list_path.append(path)
            list_ff.append(name)


## Parse .eml and Generate DF 

In [414]:

## define file start and end range 
fst = 0 
fed = 4000
fed = min(fed, len(list_ff))

## list of fields to parse from .eml , this is manual based on observation of eml file and .fields

list_fields = ['Date', 'From', 'To', 'Subject',]
list_df_cols = ['file', 'path'] + list_fields
list_df =[]


"ref :: https://www.programcreek.com/python/example/2579/email.message_from_file"
" needed try/ except as message from file failing in some instances need debug "
" used for in range and then used the num reference on path and file lists "

for num in tqdm(range(fst,fed)):
    fp = open(os.path.join(list_path[num], list_ff[num]), "r")
    try:
        msg = email.message_from_file(fp, _class=None, )
        list_op = [list_ff[num], list_path[num],]
        for lf in list_fields:
            list_op.append(msg.get_all(lf)[0])


        list_df.append(list_op)
    except:
        continue
        
    
"alt ref :: https://stackoverflow.com/questions/26309962/appending-a-list-or-series-to-a-pandas-dataframe-as-a-row"

## converting list of lists into familiar pd.dataframe with col_names 
df_list = pd.DataFrame(list_df, columns = list_df_cols)


100%|██████████| 3628/3628 [00:41<00:00, 87.93it/s] 


## Process data Engineer DF 

In [415]:
df_list.shape

(3613, 6)

In [416]:
## Processing the resultant DF to generate features needed for final alt_file name generation 

## handling Date field most effort 
"""
this took maximum time, after a lot of experiment, ended up using parser.parse in a ufunc 
apply lambda function, 
"""
df_list['Date'] =  df_list['Date'].replace({ " +\(.*$": ""}, regex=True)
df_list['Date_new'] =  df_list['Date'].apply(lambda x : str(convert_date_2_string(x)))
    

    
## parsing From and To fields, 
"ref :: https://stackoverflow.com/questions/35491274/split-a-pandas-column-of-lists-into-multiple-columns"

"""
this needed some iterations, finally, using multiple strings to split 
covert to_list and picked only 1 columns to pass into specific columns 
np.where on x_email  with series to_numpy to check for none and replace with x_name 
finally clean the output using df.filter and replace 
"""

df_list[['From_name','From_email']]  = pd.DataFrame(df_list['From'].str.lower().str.split('<|>|\,|\;').to_list())[[0,1]]
df_list[['To_name','To_email']]  = pd.DataFrame(df_list['To'].str.lower().str.split('<|>|\,|\;').to_list())[[0,1]]

df_list['To_email'] = np.where(df_list['To_email'].to_numpy() == None, df_list['To_name'], df_list['To_email'] )
df_list['From_email'] = np.where(df_list['From_email'].to_numpy() == None, df_list['From_name'], df_list['From_email'] )

df_list[list(df_list.filter(regex="To_|From_"))] = df_list.filter(regex="To_|From_").replace({"(\>|\")" : "", " " : "_",  }, regex=True)


"""
Generate formatted subject col using replace 
"""

df_list['Sub_fmt']  = df_list['Subject'].replace(
    {"\w*\:": "", "  " : " ", " ": "_", "\W" : "", "_+" : "_" },regex=True).str[0:30]


"""
Generate alt_file from a str append of multiple columns 
"""


df_list['alt_file'] = ( (df_list['Date_new']).apply(lambda x : str(x)[0:11])
                       + " " + df_list['From_email']
                       + " " + df_list['To_email']
                       + " " + df_list['Sub_fmt']
                       + " " + df_list['file']
                      )

In [417]:
## just checking the quality of the alt_file in terms of str len so that there are no name failurees 
df_list[df_list['alt_file'].str.len() > 200]

,file,path,Date,From,To,Subject,Date_new,From_name,From_email,To_name,To_email,Sub_fmt,alt_file


In [418]:
## saving the dataframe to file 
df_list.to_csv(f'{root_path}/emails_table_renew.tsv', sep='\t', index=False)

## Use Df to rename eml files 

In [419]:
## Rename files using os.rename 
"ref: https://www.geeksforgeeks.org/rename-multiple-files-using-python/ "
for nn in tqdm(range(df_list['file'].shape[0])):
    os.rename(
        os.path.join(str(df_list['path'][nn]), str(df_list['file'][nn])), 
        os.path.join(str(df_list['path'][nn]), str(df_list['alt_file'][nn])), 
             )

100%|██████████| 3613/3613 [00:00<00:00, 7680.07it/s]


In [420]:
df_list.to_csv(f'{root_path}/emails_table.tsv', sep='\t', index=False)

## End of Utility